# !!!IMPORTANT SETUP!!!

## from your GoogleDrive:
1.   create a folder ProjectAML in /content/drive/MyDrive
2.   upload in ProjectAML (the modified version of) python code @ https://github.com/shyam671/AnomalySegmentation_CourseProjectBaseCode
3. create an empty folder /ProjectAML/datasets/Validation_Dataset
4.  import manually all the test dataset in /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset from https://drive.google.com/file/d/1r2eFANvSlcUjxcerjC8l6dRa0slowMpx/view, folder must contain subfolder of the zip (FS_LostFound_full, fs_static, RoadAnomaly, RoadAnomaly21, RoadObstacles)
5. create an empty folder /ProjectAML/datasets/Train_Dataset

## NOTE: training use a high capacity RAM -> Colab Pro, but you can skip since exist a pre-trained model

# **Install required modules**

In [ ]:
#do once
!pip3 install 'torch'
!pip3 install 'torchvision'
#optimized version of Pillow
!pip3 install 'Pillow-SIMD'
!pip3 install 'numpy'
!pip3 install 'matplotlib'
#required to work, don't use --visualize
!pip3 install 'visdom'
#script to download train dataset
!pip3 install 'cityscapesscripts'
!pip3 install 'ood-metrics'
#restart after import

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.9/849.9 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Pillow-SIMD: filename=Pillow_SIMD-9.0.0.post1-cp310-cp310-linux_x86_64.whl size=1238160 sha256=8b9bfbe6046cd62c46a91923a897aae2570f4faec6031b099f49a4a482e2a456
  Stored in directory: /root/.cache/pip/wheels/5b/40/80/c3580fdb22f9cc7362b26cac344b501994785f96329a3f6b94
Successfully built Pillow-SIMD


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408199 sha256=b94e00388543cb2625acf79b780933b660669e32a89883277bc734481a64a078
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.3/473.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.3 MB/s eta 0:00:00
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=f8cbbd054cb0e5e843e1f48bebfb518d1528d91d2cdc343cede27fd2b219b5a5
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098b

# **Mount drive**

In [ ]:
import torch
#use GPU if available
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cpu' # 'cuda' or 'cpu'
print(DEVICE)

from google.colab import drive
drive.mount('/content/drive')


cuda:0
Mounted at /content/drive


# **Unmount drive**

In [ ]:
#synchronize your google drive after any modification to ProjectAML folder
drive.flush_and_unmount() #synch
drive.mount('/content/drive') #remount

Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/Validation_Dataset.zip \
 -d /content/drive/MyDrive/ProjectAML/datasets/

replace /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# **Import and reorganize cityscapes dataset**
### necessary to run IoU evaluation on val set

In [ ]:
#do once
import os
#switch to cpu for download
DEVICE = torch.device("cpu")

!rm -r /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset
!mkdir /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset

#register to https://www.cityscapes-dataset.com and login here
#241MB
!csDownload gtFine_trainvaltest.zip -d /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset
#register to https://www.cityscapes-dataset.com and login here or use saved credentials
#11GB
!csDownload leftImg8bit_trainvaltest.zip -d /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset

#unzip dataset
!unzip -q /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest.zip \
 -d /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest
!unzip -q /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest.zip \
 -d /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest



Cityscapes username or email address: maxmueh
Cityscapes password: 
Store credentials unencrypted in '/root/.local/share/cityscapesscripts/credentials.json' [y/N]: y


In [ ]:
#make sure that unzip complete successfully before executing this cell

#remove zip
!rm /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest.zip
!rm /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest.zip

In [ ]:

#move only necessary folder in /Train_Dataset
!mv /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest/gtFine \
  /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine
!mv /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest/leftImg8bit \
  /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit



In [ ]:
#make sure that mv complete successfully before executing this cell

#remove original folder (with README and other stuff)
!rm -r /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest
!rm -r /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest

#set required OS env variable to run correctly csCreateTrainIdInstanceImgs
os.environ['CITYSCAPES_DATASET'] = '/content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/'
#create label
!csCreateTrainIdLabelImgs

# return to gpu
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cpu' # 'cuda' or 'cpu'

Processing 5000 annotation files
Progress: 100.0 % 

# **START TRAINING**
## skip, trained model already exists

In [ ]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/train"
os.chdir(folder_path)

!rm -r /content/drive/MyDrive/ProjectAML/save
!mkdir /content/drive/MyDrive/ProjectAML/save

%run main.py --savedir erfnet_train_cityscape \
 --datadir /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset \
 --decoder --pretrainedEncoder /content/drive/MyDrive/ProjectAML/trained_models/erfnet_encoder_pretrained.pth.tar

#python -m visdom.server -port 8097 # run on local pc-> see result on localhost

# Eval AuPRC FPR95 on datasets

In [ ]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)
#modify parser: parse all the path elsewhere
%run eval_iou.py --datadir '/content/drive/MyDrive/ProjectAML/datasets/Train_Dataset' --subset val


Loading model: ../trained_models/erfnet.py
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
/content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit/val /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine/val
0 val/frankfurt/frankfurt_000000_000294_leftImg8bit.png
1 val/frankfurt/frankfurt_000000_000576_leftImg8bit.png
2 val/frankfurt/frankfurt_000000_001016_leftImg8bit.png
3 val/frankfurt/frankfurt_000000_001236_leftImg8bit.png
4 val/frankfurt/frankfurt_000000_001751_leftImg8bit.png
5 val/frankfurt/frankfurt_000000_002196_leftImg8bit.png
6 val/frankfurt/frankfurt_000000_002963_leftImg8bit.png
7 val/frankfurt/frankfurt_000000_003025_leftImg8bit.png
8 val/frankfurt/frankfurt_000000_003357_leftImg8bit.png
9 val/frankfurt/frankfurt_000000_003920_leftImg8bit.png
10 val/frankfurt/frankfurt_000000_004617_leftImg8bit.png
11 val/frankfurt/frankfurt_000000_005543_leftImg8bit.png
12 val/frankfurt/frankfurt_000000_005898_leftIm

# Run MSP, max logit, max entropy


In [ ]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)

print("RoadAnomaly21 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png' \
 --anomalyScore 'MSP'
print("RoadObstacle21 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp' \
 --anomalyScore 'MSP'
print("FS_LostFound_full MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png' \
 --anomalyScore 'MSP'
print("fs_static MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg' \
 --anomalyScore 'MSP'
print("RoadAnomaly MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg' \
 --anomalyScore 'MSP'

print("RoadAnomaly21 max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png'
print("RoadObstacle21 max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp'
print("FS_LostFound_full max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png'
print("fs_static max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg'
print("RoadAnomaly max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg'


print("RoadAnomaly21 max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png' \
--anomalyScore 'maxEntropy'
print("RoadObstacle21 max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp' \
--anomalyScore 'maxEntropy'
print("FS_LostFound_full max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png' \
--anomalyScore 'maxEntropy'
print("fs_static max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg' \
--anomalyScore 'maxEntropy'
print("RoadAnomaly max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg' \
--anomalyScore 'maxEntropy'

RoadAnomaly21 MSP
Loading model: /content/drive/MyDrive/ProjectAML/eval/erfnet.py
Loading weights: /content/drive/MyDrive/ProjectAML/save/erfnet_train_cityscape/model_best.pth
Model and weights LOADED successfully
AUPRC score: 21.972433783659287
FPR@TPR95: 76.73269273148826
RoadObstacle21 MSP
Loading model: /content/drive/MyDrive/ProjectAML/eval/erfnet.py
Loading weights: /content/drive/MyDrive/ProjectAML/save/erfnet_train_cityscape/model_best.pth
Model and weights LOADED successfully
AUPRC score: 0.8288415750151416
FPR@TPR95: 74.04122019222062
FS_LostFound_full MSP
Loading model: /content/drive/MyDrive/ProjectAML/eval/erfnet.py
Loading weights: /content/drive/MyDrive/ProjectAML/save/erfnet_train_cityscape/model_best.pth
Model and weights LOADED successfully
AUPRC score: 1.4542583059342993
FPR@TPR95: 70.20506574806731
fs_static MSP
Loading model: /content/drive/MyDrive/ProjectAML/eval/erfnet.py
Loading weights: /content/drive/MyDrive/ProjectAML/save/erfnet_train_cityscape/model_best.pt

# Run temperature scaled MSP

In [ ]:
import os
from IPython import get_ipython
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)

def run_command(command):
    result = get_ipython().system(command)
    if result != 0:
        print(f"Error executing command: {command}")


for t in [0.5, 0.75, 1.1]:

    # Define the commands in a list
    commands = [
        f'echo Running RoadAnomaly21 temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png --anomalyScore tempMSP --temp {t}",
        f'echo Running RoadObstacle21 temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp --anomalyScore 'tempMSP' --temp {t}",
        f'echo Running FS_LostFound_full temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png --anomalyScore 'tempMSP' --temp {t}",
        f'echo Running fs_static temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg --anomalyScore 'tempMSP' --temp {t}",
        f'echo Running RoadAnomaly temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg --anomalyScore 'tempMSP' --temp {t}"
    ]

    # Execute the commands using subprocess.run
    for command in commands:
        run_command(command)


Running RoadAnomaly21 temp=0.5 MSP
Error executing command: echo Running RoadAnomaly21 temp=0.5 MSP
Loading model: /content/drive/MyDrive/ProjectAML/eval/erfnet.py
Loading weights: /content/drive/MyDrive/ProjectAML/save/erfnet_train_cityscape/model_best.pth
Model and weights LOADED successfully
AUPRC score: 20.21395272072145
FPR@TPR95: 76.6010547898038
Error executing command: python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png --anomalyScore tempMSP --temp 0.5
Running RoadObstacle21 temp=0.5 MSP
Error executing command: echo Running RoadObstacle21 temp=0.5 MSP
Loading model: /content/drive/MyDrive/ProjectAML/eval/erfnet.py
Loading weights: /content/drive/MyDrive/ProjectAML/save/erfnet_train_cityscape/model_best.pth
Model and weights LOADED successfully
AUPRC score: 0.8385069624363087
FPR@TPR95: 73.96382700503007
Error executing command: python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validati

# Grid search of best temperature for MSP

In [ ]:
import os
from IPython import get_ipython
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)

def run_command(command):
    result = get_ipython().system(command)
    if result != 0:
        print(f"Error executing command: {command}")


for i in range(4, -5, -1):
    t = format(f"{10**i}")

    # Define the commands in a list
    commands = [
        f'echo Running RoadAnomaly21 temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png --anomalyScore tempMSP --temp {t}",
        f'echo Running RoadObstacle21 temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp --anomalyScore 'tempMSP' --temp {t}",
        f'echo Running FS_LostFound_full temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png --anomalyScore 'tempMSP' --temp {t}",
        f'echo Running fs_static temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg --anomalyScore 'tempMSP' --temp {t}",
        f'echo Running RoadAnomaly temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg --anomalyScore 'tempMSP' --temp {t}"
    ]

    # Execute the commands using subprocess.run
    for command in commands:
        run_command(command)



Running RoadAnomaly21 temp=10000 MSP
Error executing command: echo Running RoadAnomaly21 temp=10000 MSP
Loading model: /content/drive/MyDrive/ProjectAML/eval/erfnet.py
Loading weights: /content/drive/MyDrive/ProjectAML/save/erfnet_train_cityscape/model_best.pth
Model and weights LOADED successfully
AUPRC score: 23.019983102662405
FPR@TPR95: 92.84313633424632
Error executing command: python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png --anomalyScore tempMSP --temp 10000
Running RoadObstacle21 temp=10000 MSP
Error executing command: echo Running RoadObstacle21 temp=10000 MSP
Loading model: /content/drive/MyDrive/ProjectAML/eval/erfnet.py
Loading weights: /content/drive/MyDrive/ProjectAML/save/erfnet_train_cityscape/model_best.pth
Model and weights LOADED successfully
AUPRC score: 0.9338655538544914
FPR@TPR95: 79.56166884871465
Error executing command: python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datas

# Void classifier (background score)

In [ ]:
#redo training (see Slack)
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/train"
os.chdir(folder_path)

!rm -r /content/drive/MyDrive/ProjectAML/save
!mkdir /content/drive/MyDrive/ProjectAML/save

%run main.py --savedir erfnet_train_cityscape_20class \
 --datadir /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset \
 --decoder --pretrainedEncoder /content/drive/MyDrive/ProjectAML/trained_models/erfnet_encoder_pretrained.pth.tar
#python -m visdom.server -port 8097 # run on local pc-> see result on localhost



========== DECODER TRAINING ===========
Loading encoder pretrained in imagenet
/content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit/train
/content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit/val
<class '__main__.CrossEntropyLoss2d'>
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.0005029990024359241
loss: 3.046 (epoch: 1, step: 0) // Avg time/img: 0.4348 s
loss: 2.654 (epoch: 1, step: 50) // Avg time/img: 0.1606 s
loss: 2.271 (epoch: 1, step: 100) // Avg time/img: 0.1619 s
loss: 1.995 (epoch: 1, step: 150) // Avg time/img: 0.1619 s
loss: 1.781 (epoch: 1, step: 200) // Avg time/img: 0.1622 s
loss: 1.631 (epoch: 1, step: 250) // Avg time/img: 0.1622 s
loss: 1.517 (epoch: 1, step: 300) // Avg time/img: 0.1623 s
loss: 1.425 (epoch: 1, step: 350) // Avg time/img: 0.1624 s
loss: 1.353 (epoch: 1, step: 400) // Avg time/img: 0.1625 s
loss: 1.293 (epoch: 1, step: 450) // Avg time/img: 0.1625 s
----- VALIDATING - EPOCH 1 -----


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/content/drive/MyDrive/ProjectAML/train/main.py:294: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
/content/drive/MyDrive/ProjectAML/train/main.py:295: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  targets = Variable(labels, volatile=True)


VAL loss: 0.5793 (epoch: 1, step: 0) // Avg time/img: 0.0464 s
VAL loss: 0.8652 (epoch: 1, step: 50) // Avg time/img: 0.0384 s
EPOCH IoU on VAL set:  25.97 %
Saving model as best
save: ../save/erfnet_train_cityscape_20class/model_best.pth (epoch: 1)
----- TRAINING - EPOCH 2 -----
LEARNING RATE:  0.0005
loss: 0.7376 (epoch: 2, step: 0) // Avg time/img: 0.1634 s
loss: 0.728 (epoch: 2, step: 50) // Avg time/img: 0.1629 s
loss: 0.7297 (epoch: 2, step: 100) // Avg time/img: 0.1627 s
loss: 0.7112 (epoch: 2, step: 150) // Avg time/img: 0.1627 s
loss: 0.686 (epoch: 2, step: 200) // Avg time/img: 0.1627 s
loss: 0.6715 (epoch: 2, step: 250) // Avg time/img: 0.1627 s
loss: 0.6571 (epoch: 2, step: 300) // Avg time/img: 0.1627 s
loss: 0.6502 (epoch: 2, step: 350) // Avg time/img: 0.1627 s
loss: 0.6381 (epoch: 2, step: 400) // Avg time/img: 0.1628 s
loss: 0.6267 (epoch: 2, step: 450) // Avg time/img: 0.1628 s
----- VALIDATING - EPOCH 2 -----
VAL loss: 0.3254 (epoch: 2, step: 0) // Avg time/img: 0.03

In [ ]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)



print("RoadAnomaly21 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png' \
 --anomalyScore 'MSP' --void "yes"
print("RoadObstacle21 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp' \
 --anomalyScore 'MSP' --void "yes"
print("FS_LostFound_full MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png' \
 --anomalyScore 'MSP' --void "yes"
print("fs_static MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg' \
 --anomalyScore 'MSP' --void "yes"
print("RoadAnomaly MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg' \
 --anomalyScore 'MSP' --void "yes"

RoadAnomaly21 MSP
Loading model: /content/drive/MyDrive/ProjectAML/eval/erfnet.py
Loading weights: /content/drive/MyDrive/ProjectAML/save/erfnet_train_cityscape/model_best.pth
Model and weights LOADED successfully
AUPRC score: 9.82709173636973
FPR@TPR95: 93.95498566250068
RoadObstacle21 MSP
Loading model: /content/drive/MyDrive/ProjectAML/eval/erfnet.py
Loading weights: /content/drive/MyDrive/ProjectAML/save/erfnet_train_cityscape/model_best.pth
Model and weights LOADED successfully
AUPRC score: 0.41282020754768456
FPR@TPR95: 96.73133286263025
FS_LostFound_full MSP
Loading model: /content/drive/MyDrive/ProjectAML/eval/erfnet.py
Loading weights: /content/drive/MyDrive/ProjectAML/save/erfnet_train_cityscape/model_best.pth
Model and weights LOADED successfully
AUPRC score: 0.19132385049389986
FPR@TPR95: 96.4291483390457
fs_static MSP
Loading model: /content/drive/MyDrive/ProjectAML/eval/erfnet.py
Loading weights: /content/drive/MyDrive/ProjectAML/save/erfnet_train_cityscape/model_best.pth